In [1]:
%%capture
import os

# check if this is a colab notebook and clone the repo if it is
if 'COLAB_GPU' in os.environ:
    %cd /content/
    !git clone https://github.com/Enver-group/twitch-web-analytics
    %cd twitch-web-analytics
else:
    os.chdir('..')

!pip install -r requirements.txt

!pip install -e .

In [2]:
#autoreload modules
%load_ext autoreload
%autoreload 2

from src.user import User

In [3]:
ibai = User.from_name("ibai")
ibai_follows = ibai.follows
print(f"{ibai.name.capitalize()} follows {len(ibai_follows)} users")

Ibai follows 155 users


In [4]:
users_that_ibai_follows = User.get_users(ibai_follows)
print("%s users retrieved from the list of follows"%len(users_that_ibai_follows))

155 users retrieved from the list of follows


In [5]:
import pandas as pd
df = pd.DataFrame([ibai]+users_that_ibai_follows)
df

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,None,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286992630,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"[35980866, 145908612, 205218019, 91136321, 524..."
1,35980866,Perxitaa,None,partner,Troll de Youtube http://www.youtube.com/user/p...,es,Grand Theft Auto V,38783064,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-09-05T22:12:47Z,None
2,145908612,momoladinastia,None,partner,"Siempre al pie para los mios, Stream #ATR re b...",es,Just Chatting,11215752,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-01-25T01:52:25Z,None
3,205218019,el_yuste,None,partner,Lo que viene a ser Yuste diciendo sus cosas.,es,Just Chatting,1095637,https://static-cdn.jtvnw.net/jtv_user_pictures...,2018-03-13T21:58:12Z,None
4,91136321,karchez,None,partner,"Soy Karchez, me gusta llevar corona y hago el ...",es,Just Chatting,13131113,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-05-15T21:35:29Z,None
...,...,...,...,...,...,...,...,...,...,...,...
151,38446500,G4G_REVENANT,None,partner,Tony el Parrys en acción,es,League of Legends,30779077,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-12-13T22:29:18Z,None
152,25058448,orslok,None,partner,____,es,The Legend of Zelda: Breath of the Wild,14405420,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-09-25T09:46:19Z,None
153,47919664,champi14,None,partner,"Color caster de League of Legends en LVP, diam...",es,League of Legends,1413087,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-08-21T16:28:59Z,None
154,102404582,silerr8,None,partner,Midlaner de TEAM QUESO y campeon del SOLOQCHAL...,es,League of Legends,2940632,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-09-17T18:41:42Z,None


In [6]:
from src.data import make_data_from_root_user

# Obtiene datos de cada usuario del arbol empezando por Ibai y sus follows
# cada iteracion siguiente coge un usuario del arbol aleatorio para añadir sus follows 
# y asi hasta llegar a max_users o que se interrumpa manualmente. 
# Cada 50 iteraciones guarda el progreso actual en un archivo csv
make_data_from_root_user("ibai",output_file="data/data.csv",max_users=200)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: ibai
INFO:src.data.make_dataset:New users will be fetched until 200 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 140 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:max_users reached. 299 users were retrieved.
INFO:src.data.make_dataset:writing dataset to file before stopping...


,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,None,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286992630,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"[35980866, 145908612, 205218019, 91136321, 524..."
1,269641929,PiezasMFT,None,partner,Piezas. Murcia Finest. Embajador de x6tence. D...,es,Just Chatting,2439534,https://static-cdn.jtvnw.net/jtv_user_pictures...,2018-10-26T17:33:04Z,"[447059771, 208813061, 167100879, 235551224, 1..."
2,192763971,Morcytwitch,None,,Y YO PREOCUPAO !!!! Pan Pijo y Habas !!!!,es,FIFA 22,2297,https://static-cdn.jtvnw.net/jtv_user_pictures...,2018-01-25T10:27:15Z,None
3,74426799,GUANYAR,None,partner,El favorito de ninguna,es,Just Chatting,5333243,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-11-03T21:37:54Z,None
4,523784924,BelenJuradoMusic,None,partner,"Hola! Soy cantante 🎙, pianista 🎹 y compositora...",es,Music,767115,https://static-cdn.jtvnw.net/jtv_user_pictures...,2020-05-01T17:42:57Z,None
...,...,...,...,...,...,...,...,...,...,...,...
294,104725246,DrouxTV,None,affiliate,Roleplayer en OverlifeRP como Héctor Reyes. Cr...,es,Grand Theft Auto V,54993,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-10-19T13:34:30Z,None
295,30357893,Werlyb,None,partner,,es,League of Legends,28553607,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-05-07T16:02:16Z,None
296,66399374,galletas,None,partner,Me enfado mucho pero es todo en clave de humor.,es,Street Fighter V,270725,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-07-15T15:32:59Z,None
297,66727526,gravesen_1,None,partner,(This user has no bio),es,FIFA 22,1586989,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-07-19T18:31:12Z,None


In [7]:
data = pd.read_csv("data/data.csv")
data.drop_duplicates("id", keep='first',inplace=True)
data.fillna({"broadcaster_type":"user"},inplace=True)
data = data.loc[data['lang']=='es']

In [8]:
data.sort_values(by='view_count', ascending=False)

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,NaN,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286992630,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"['35980866', '145908612', '205218019', '911363..."
12,22346597,LVPes,NaN,partner,NaN,es,Special Events,227606755,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,NaN
144,39276140,Rubius,NaN,partner,se contar hasta patata,es,Propnight,222836918,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12T18:46:56Z,NaN
189,459331509,auronplay,NaN,partner,feliz pero no mucho,es,Minecraft,212162716,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03T14:02:49Z,NaN
147,48878319,TheGrefg,NaN,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Just Chatting,197835374,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12T00:45:27Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...
98,644121740,TodosSomosSantiuve,NaN,affiliate,"Santiuve es un chaval que rapea, y que al pare...",es,Papetura,355,https://static-cdn.jtvnw.net/jtv_user_pictures...,2021-01-31T11:52:45Z,NaN
170,425402130,KikoVidalPro_NoGimmicks,NaN,user,"No se hacer esto y por eso me gusta, cuando ap...",es,Travel & Outdoors,283,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-03-23T16:29:54Z,NaN
111,93150630,BurzeBeats,NaN,user,NaN,es,Rocket League,243,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-06-10T11:50:08Z,NaN
159,88572935,leprosyfoxtrot,NaN,user,NaN,es,League of Legends,153,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-04-14T20:23:12Z,NaN


In [9]:
from src.data import make_data_from_root_user
from src.data import extract_follows_from_users_df

max_users = 100000
top_users = 5000
initial_user = 'ibai' #Starting point for the random algorithm

make_data_from_root_user(initial_user,output_file="data/data.csv",max_users=max_users)
data = pd.read_csv("data/data.csv")
data.drop_duplicates("id", keep='first',inplace=True) #Remove posible duplicates 
data.fillna({"broadcaster_type":"user"},inplace=True) #Fill nans with 'user' tag
data = data.loc[data['lang']=='es'] #Get just hispanian people
data = data.loc[data['broadcaster_type']!='user'] #Getting rid of not broadcasters/affiliates
data.sort_values(by='view_count', ascending=False,inplace=True) #Sorting by the number of view count

top_df = data.head(top_users).copy()

top_df = extract_follows_from_users_df(top_df)

for i in range(0,top_users):
    data.iloc[i] = top_df.iloc[i]

data.to_csv(r'data/data_davimen.csv',index=False,header=True)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: ibai
INFO:src.data.make_dataset:New users will be fetched until 100000 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 140 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:Iteration 11 - 1594 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:Iteration 21 - 3202 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:Iteration 31 - 4547 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data.csv
INFO:src.data.make_dataset:Iteration 41 - 5224 users have been retrieved

'access_token'


INFO:src.data.make_dataset:420/5000 have been processed.
INFO:src.data.make_dataset:430/5000 have been processed.
INFO:src.data.make_dataset:440/5000 have been processed.
INFO:src.data.make_dataset:450/5000 have been processed.
INFO:src.data.make_dataset:460/5000 have been processed.
INFO:src.data.make_dataset:470/5000 have been processed.
INFO:src.data.make_dataset:480/5000 have been processed.
INFO:src.data.make_dataset:490/5000 have been processed.
INFO:src.data.make_dataset:500/5000 have been processed.
INFO:src.data.make_dataset:510/5000 have been processed.
INFO:src.data.make_dataset:520/5000 have been processed.
INFO:src.data.make_dataset:530/5000 have been processed.
INFO:src.data.make_dataset:540/5000 have been processed.
INFO:src.data.make_dataset:550/5000 have been processed.
INFO:src.data.make_dataset:560/5000 have been processed.
INFO:src.data.make_dataset:570/5000 have been processed.
INFO:src.data.make_dataset:580/5000 have been processed.
INFO:src.data.make_dataset:590/

In [61]:
data = pd.read_csv("data/data_ibai.csv")
data.head(100)

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,NaN,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286992630,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"['35980866', '145908612', '205218019', '911363..."
1,22346597,LVPes,NaN,partner,NaN,es,Special Events,227606755,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,"['572984021', '113366816', '420395016', '23286..."
2,39276140,Rubius,NaN,partner,se contar hasta patata,es,Propnight,222893232,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12T18:46:56Z,"['277057209', '603776329', '91136321', '198363..."
3,459331509,auronplay,NaN,partner,feliz pero no mucho,es,Minecraft,212164590,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03T14:02:49Z,"['210708721', '431460701', '77649106', '130065..."
4,48878319,TheGrefg,NaN,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Five Nights at Freddy's,197871081,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12T00:45:27Z,"['143776262', '675347177', '33734881', '248222..."
...,...,...,...,...,...,...,...,...,...,...,...
95,41879247,ernesBarbeQ,NaN,partner,NaN,es,Teamfight Tactics,15475531,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-03-30T15:15:51Z,"['63453024', '106867975', '58486016', '1834952..."
96,199811071,Robleis,NaN,partner,do what u want.,es,Just Chatting,15266380,https://static-cdn.jtvnw.net/jtv_user_pictures...,2018-03-01T16:58:34Z,"['118220323', '109492660', '157658336', '91136..."
97,32505769,MrKeroro10,NaN,partner,NaN,es,Fall Guys: Ultimate Knockout,15117966,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-07-28T19:00:10Z,"['133955534', '69564524', '522329407', '109492..."
98,175779148,LuisOrmenoa27,NaN,partner,"Stream todos los días jugando Free Fire, reacc...",es,Just Chatting,15040970,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-09-28T15:13:02Z,"['584021851', '174181287', '138094916', '22665..."


In [15]:
from src.data import make_data_from_root_user
from src.data import extract_follows_from_users_df

max_users = 24
top_users = 17
initial_user = 'davimenxpro' #Starting point for the random algorithm

make_data_from_root_user(initial_user,output_file="data/data_manual.csv",max_users=max_users)
data = pd.read_csv("data/data_manual.csv")
data.drop_duplicates("id", keep='first',inplace=True) #Remove posible duplicates 
data.fillna({"broadcaster_type":"user"},inplace=True) #Fill nans with 'user' tag
data = data.loc[data['lang']=='es'] #Get just hispanian people
data = data.loc[data['broadcaster_type']!='user'] #Getting rid of not broadcasters/affiliates
data.sort_values(by='view_count', ascending=False,inplace=True) #Sorting by the number of view count

top_df = data.head(top_users).copy()

top_df = extract_follows_from_users_df(top_df)

for i in range(0,top_users):
    data.iloc[i] = top_df.iloc[i]

data.to_csv(r'data/data_davimen_manual.csv',index=False,header=True)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: davimenxpro
INFO:src.data.make_dataset:New users will be fetched until 24 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 24 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/data_manual.csv
INFO:src.data.make_dataset:max_users reached. 24 users were retrieved.
INFO:src.data.make_dataset:writing dataset to file before stopping...
INFO:src.data.make_dataset:1/17 have been processed.
INFO:src.data.make_dataset:10/17 have been processed.


In [22]:
data2 = pd.read_csv("data/data_davimen_manual.csv")
data.head(17)

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,NaN,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286992630,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"['35980866', '145908612', '205218019', '911363..."
1,22346597,LVPes,NaN,partner,NaN,es,Special Events,227606755,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,"['572984021', '113366816', '420395016', '23286..."
2,39276140,Rubius,NaN,partner,se contar hasta patata,es,Propnight,222893232,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12T18:46:56Z,"['277057209', '603776329', '91136321', '198363..."
3,459331509,auronplay,NaN,partner,feliz pero no mucho,es,Minecraft,212164590,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03T14:02:49Z,"['210708721', '431460701', '77649106', '130065..."
4,48878319,TheGrefg,NaN,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Five Nights at Freddy's,197871081,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12T00:45:27Z,"['143776262', '675347177', '33734881', '248222..."
5,121510236,juansguarnizo,NaN,partner,"Juan Sebastián Guarnizo Algarra, más conocido ...",es,Just Chatting,127964214,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-04-11T03:34:11Z,"['43419527', '517536651', '476005292', '109492..."
6,30610294,Jaidefinichon,NaN,partner,NaN,es,Marbles On Stream,105015714,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-05-17T22:18:23Z,"['459331509', '48878319', '500128827', '452785..."
7,123922797,IamCristinini,NaN,partner,📺 Presentadora de TV || 🎙 Doblaje || 🎮 Videoju...,es,Minecraft,83091846,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-05-10T10:15:59Z,"['36061968', '495899004', '280408230', '431146..."
8,36473331,coscu,NaN,partner,"Martin Pérez Disalvo alias Coscu, embajador de...",es,Tetris Effect: Connected,80752758,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-09-26T21:50:41Z,"['144788538', '150935014', '113831945', '41497..."
9,44880944,ElmiilloR,NaN,partner,CHALLENGER RANK 1 EN TODOS LOS GAMES JAMÁS CRA...,es,Slots,78625449,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-06-17T18:54:08Z,"['509100013', '491811504', '127665820', '72550..."


In [71]:
data = pd.read_csv("data/data_ibai.csv")
df_user_follows = data[-data['user_follows'].isnull()].sort_values(by='view_count', ascending=False,ignore_index=True) #Getting just the people with user_follows
df_user_follows.head(50)

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,83232866,ibai,NaN,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286992630,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"['35980866', '145908612', '205218019', '911363..."
1,22346597,LVPes,NaN,partner,NaN,es,Special Events,227606755,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,"['572984021', '113366816', '420395016', '23286..."
2,39276140,Rubius,NaN,partner,se contar hasta patata,es,Propnight,222893232,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12T18:46:56Z,"['277057209', '603776329', '91136321', '198363..."
3,459331509,auronplay,NaN,partner,feliz pero no mucho,es,Minecraft,212164590,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03T14:02:49Z,"['210708721', '431460701', '77649106', '130065..."
4,48878319,TheGrefg,NaN,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Five Nights at Freddy's,197871081,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12T00:45:27Z,"['143776262', '675347177', '33734881', '248222..."
5,121510236,juansguarnizo,NaN,partner,"Juan Sebastián Guarnizo Algarra, más conocido ...",es,Just Chatting,127964214,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-04-11T03:34:11Z,"['43419527', '517536651', '476005292', '109492..."
6,30610294,Jaidefinichon,NaN,partner,NaN,es,Marbles On Stream,105015714,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-05-17T22:18:23Z,"['459331509', '48878319', '500128827', '452785..."
7,123922797,IamCristinini,NaN,partner,📺 Presentadora de TV || 🎙 Doblaje || 🎮 Videoju...,es,Minecraft,83091846,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-05-10T10:15:59Z,"['36061968', '495899004', '280408230', '431146..."
8,36473331,coscu,NaN,partner,"Martin Pérez Disalvo alias Coscu, embajador de...",es,Tetris Effect: Connected,80752758,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-09-26T21:50:41Z,"['144788538', '150935014', '113831945', '41497..."
9,44880944,ElmiilloR,NaN,partner,CHALLENGER RANK 1 EN TODOS LOS GAMES JAMÁS CRA...,es,Slots,78625449,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-06-17T18:54:08Z,"['509100013', '491811504', '127665820', '72550..."
